In [2]:
# -*- coding: utf8 -*-

from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals

import math

try:
    import numpy
except ImportError:
    numpy = None

from sumy.summarizers._summarizer import AbstractSummarizer
from sumy._compat import Counter


class BetaSummarizer(AbstractSummarizer):
    """
    LexRank: Graph-based Centrality as Salience in Text Summarization
    Source: http://tangra.si.umich.edu/~radev/lexrank/lexrank.pdf
    """
    threshold = 0.1
    epsilon = 0.1
    _stop_words = frozenset()

    @property
    def stop_words(self):
        return self._stop_words

    @stop_words.setter
    def stop_words(self, words):
        self._stop_words = frozenset(map(self.normalize_word, words))

    def __call__(self, document, sentences_count):
        self._ensure_dependencies_installed()

        sentences_words = [self._to_words_set(s) for s in document.sentences]
        if not sentences_words:
            return tuple()

        tf_metrics = self._compute_tf(sentences_words)
        idf_metrics = self._compute_idf(sentences_words)

        matrix = self._create_matrix(sentences_words, self.threshold, tf_metrics, idf_metrics)
        scores = self.power_method(matrix, self.epsilon)
        ratings = dict(zip(document.sentences, scores))

        return self._get_best_sentences(document.sentences, sentences_count, ratings)

    @staticmethod
    def _ensure_dependencies_installed():
        if numpy is None:
            raise ValueError("LexRank summarizer requires NumPy. Please, install it by command 'pip install numpy'.")

    def _to_words_set(self, sentence):
        words = map(self.normalize_word, sentence.words)
        return [self.stem_word(w) for w in words if w not in self._stop_words]

    def _compute_tf(self, sentences):
        tf_values = map(Counter, sentences)

        tf_metrics = []
        for sentence in tf_values:
            metrics = {}
            max_tf = self._find_tf_max(sentence)

            for term, tf in sentence.items():
                metrics[term] = tf / max_tf

            tf_metrics.append(metrics)

        return tf_metrics

    @staticmethod
    def _find_tf_max(terms):
        return max(terms.values()) if terms else 1

    @staticmethod
    def _compute_idf(sentences):
        idf_metrics = {}
        sentences_count = len(sentences)

        for sentence in sentences:
            for term in sentence:
                if term not in idf_metrics:
                    n_j = sum(1 for s in sentences if term in s)
                    idf_metrics[term] = math.log(sentences_count / (1 + n_j))

        return idf_metrics

    def _create_matrix(self, sentences, threshold, tf_metrics, idf_metrics):
        """
        Creates matrix of shape |sentences|×|sentences|.
        """
        # create matrix |sentences|×|sentences| filled with zeroes
        sentences_count = len(sentences)
        matrix = numpy.zeros((sentences_count, sentences_count))
        degrees = numpy.zeros((sentences_count, ))

        for row, (sentence1, tf1) in enumerate(zip(sentences, tf_metrics)):
            for col, (sentence2, tf2) in enumerate(zip(sentences, tf_metrics)):
                matrix[row, col] = self.compute_distance(sentence1, sentence2, tf1, tf2, idf_metrics)

                if matrix[row, col] > threshold:
                    matrix[row, col] = 1.0
                    degrees[row] += 1
                else:
                    matrix[row, col] = 0

        for row in range(sentences_count):
            for col in range(sentences_count):
                if degrees[row] == 0:
                    degrees[row] = 1

                matrix[row][col] = matrix[row][col] / degrees[row]

        return matrix

    #@staticmethod

    @staticmethod
    def power_method(matrix, epsilon):
        transposed_matrix = matrix.T
        sentences_count = len(matrix)
        p_vector = numpy.array([1.0 / sentences_count] * sentences_count)
        lambda_val = 1.0

        while lambda_val > epsilon:
            next_p = numpy.dot(transposed_matrix, p_vector)
            lambda_val = numpy.linalg.norm(numpy.subtract(next_p, p_vector))
            p_vector = next_p

        return p_vector
    @staticmethod
    def compute_distance(sentence1, sentence2, tf1, tf2, idf_metrics):
        common_words = frozenset(sentence1) & frozenset(sentence2)

        numerator = 0.0
        for term in common_words:
            numerator += tf1[term]*tf2[term] * idf_metrics[term]**2

        denominator1 = sum((tf1[t]*idf_metrics[t])**2 for t in sentence1)
        denominator2 = sum((tf2[t]*idf_metrics[t])**2 for t in sentence2)

        if denominator1 > 0 and denominator2 > 0:
            return numerator / (math.sqrt(denominator1) * math.sqrt(denominator2))
        else:
            return 0.0   

In [6]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

LANGUAGE = "english"
SENTENCES_COUNT = 3

# parser = PlaintextParser.from_file("experimentos/01/C1_original.txt", Tokenizer(LANGUAGE))
parser = PlaintextParser.from_file("output/experiments/C49_Cotidiano_VaiaLula/original.txt", Tokenizer(LANGUAGE))
stemmer = Stemmer(LANGUAGE)
    
summarizer = BetaSummarizer(stemmer)
summarizer.stop_words = get_stop_words(LANGUAGE)

for sentence in summarizer(parser.document, SENTENCES_COUNT):
    print(sentence)

Lula diz que vaia na abertura do Pan é 'reação do ser humano'O presidente Luiz Inácio Lula da Silva classificou de reação do ser humano as vaias que recebeu, na última sexta-feira, durante a abertura dos Jogos Pan-Americanos do Rio, no Maracanã.
O Rio de Janeiro a gente poderia dizer continua lindo e merece que o governo federal faça o que for possível para o Rio de Janeiro. Lula diz que ficou triste com vaias durante abertura do PanPresidente diz que vaia na abertura do Pan não muda relação do governo com RioO presidente Luiz Inácio Lula da Silva afirmou nesta segunda-feira, durante o programa de rádio "Café com o Presidente", que ficou triste com as vaias que recebeu durante a abertura oficial da 15ª edição dos Jogos Pan-Americanos, realizada no estádio do Maracanã, no Rio de Janeiro.
Eu tenho certeza de que não é esse o pensamento do Rio de Janeiro.


In [11]:
import os, fnmatch
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

LANGUAGE = "english"
rootdir = 'output'

for subdirs, dirs, files in os.walk(rootdir):
  for dir in dirs:
    for file in os.listdir(os.path.join(subdirs, dir)):
      if fnmatch.fnmatch(file, '*original.txt'): 
        line = open(os.path.join(subdirs, dir, 'sumario_automatico_CSTSumm.txt')).read().decode('latin-1')
        SENTENCES_COUNT = line.count('.')
        parser = PlaintextParser.from_file(os.path.join(subdirs, dir, 'original.txt'), Tokenizer(LANGUAGE))
        stemmer = Stemmer(LANGUAGE)
        summarizer = BetaSummarizer(stemmer)
        summarizer.stop_words = get_stop_words(LANGUAGE)
        print("\n"+os.path.join(subdirs, dir, 'original.txt'))
        f = open(os.path.join(subdirs, dir, 'lexrank_summ.txt'), 'w+')
        for sentence in summarizer(parser.document, SENTENCES_COUNT):
          print(str(sentence))
          f.write(str(sentence))


output/experiments/C16_Politica_Sanguessugas/original.txt
Parlamentares sanguessugas têm até a meia-noite de hoje para renunciarTermina hoje, às 20 horas, o prazo para que os deputados acusados de participar do esquema dos sanguessugas renunciem para escapar da abertura de processo por quebra de decoro parlamentar.
O horário-limite para que o parlamentar renuncie - 20 horas - foi estabelecido pela direção da Câmara a fim de que o ato seja oficializado com a sua publicação já no Diário Oficial do Congresso de amanhã.
Na terça-feira, dia 22, o presidente do Conselho de Ética, Ricardo Izar (PTB-SP), deve começar a abrir processos contra os parlamentares acusados que permanecerem em seus cargos.
Dos 66 congressistas ligados a igrejas evangélicas, 23 --mais de um terço da bancada-- estão envolvidos nas irregularidades e tiveram a cassação de seus mandatos sugerida.
As renúncias têm que ser publicadas até terça-feira, quando o presidente do Conselho de Ética, deputado Ricardo Izar (PTB-SP),